# Proyecto 2 entrega #3

## Implementación del modelo de Naive Bayes

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar los datos
data = pd.read_csv('train.csv')

# Convertir la variable objetivo en categorías
num_bins = 3  # Número de categorías
labels = ['bajo', 'medio', 'alto']
data['SalePrice_cat'] = pd.qcut(data['SalePrice'], q=num_bins, labels=labels)

# Seleccionar características y variable objetivo
X = data.drop(['SalePrice', 'SalePrice_cat'], axis=1)
y = data['SalePrice_cat']

# Convertir variables categóricas en numéricas
X = pd.get_dummies(X, drop_first=True)

# Manejo de valores faltantes
X.fillna(X.mean(), inplace=True)  # Rellenar valores numéricos con la media

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Realizar predicciones
y_pred = nb_model.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Precisión del modelo: {accuracy:.4f}')
print('Matriz de confusión:\n', conf_matrix)
print('Reporte de clasificación:\n', class_report)




Precisión del modelo: 0.7637
Matriz de confusión:
 [[ 82   3   9]
 [  0 106   4]
 [ 24  29  35]]
Reporte de clasificación:
               precision    recall  f1-score   support

        alto       0.77      0.87      0.82        94
        bajo       0.77      0.96      0.85       110
       medio       0.73      0.40      0.51        88

    accuracy                           0.76       292
   macro avg       0.76      0.74      0.73       292
weighted avg       0.76      0.76      0.74       292



## Informe

### Análisis del modelo
* **Precisión Global**: 76.37% de precisión significa que el modelo claasificó correctamente alrededor del 76% de los valores conjuntos de prueba. Para un modelo de Naive Bayes, es un resultado aceptable.
* **Matriz de confusión**: 
    * Clase **alto**: 82 predicciones correctas, 12 incorrectas
    * Clase **media**: Buena precisión y recall alto, lo que significa que casi todos los casos reales de "bajo" fueron clasificados correctamente
    * Clase **medio**: Tiene la peor rendimiento con recall de 40% indicando que el modelo tiene problemas para identificar la categoría.
### Comparación de los modelos
* **Modelo Regresión Lineal**: Tiene un 79.9% de precisión, tiene los coeficientes más estables.
* **Modelo Árbol de Regresión**: Precisión de 79.58%, este siendo de produndidad 10.
* **Modelo Naive Bayes**: Precisión de 76.37%, valor aceptable

En general el modelo de regresión lineal ajustado que se hizo con anterioridad sigue siendo el mejor modelo para la predicción.